In [ ]:
import os, math, time
import numpy as np
import pandas as pd
import yfinance as yf
from fredapi import Fred
import matplotlib.pyplot as plt

# ---------------- Settings ----------------
START = "2000-01-01"   # start at Jan 2000 (IYR inception; "pre-2000 onwards")
END   = None           # None = today
FRED_API_KEY = os.getenv("FRED_API_KEY", "ba106f8e21396ed82fb009686affa297")
RETRY_N = 3
RETRY_SLEEP = 2.0
# ------------------------------------------

np.set_printoptions(suppress=True, linewidth=140)

def dl_yf(ticker, interval="1mo"):
    """Download monthly total-return-ish prices for ETFs/indices from Yahoo with retries."""
    s = pd.Series(dtype=float, name=ticker)
    for _ in range(RETRY_N):
        try:
            df = yf.download(ticker, start=START, end=END, interval=interval,
                             auto_adjust=True, progress=False, threads=False)
            if not df.empty and "Adj Close" in df.columns:
                s = df["Adj Close"].dropna().rename(ticker)
                break
        except Exception:
            pass
        time.sleep(RETRY_SLEEP)
    return s

def monthly_returns_from_prices(px):
    px = px.resample("M").last().dropna(how="all")
    return px.pct_change().dropna(how="all")

# --- replace your get_rf_monthly() with this ---
def get_rf_monthly(fred_api_key=None, rets_index=None):
    """
    Returns a monthly risk-free return series aligned to rets_index if provided.
    Priority:
      1) FRED TB3MS (monthly)
      2) FRED DTB3 (daily 13-week) -> resampled to month-end
      3) Yahoo ^IRX (13-week T-bill) -> resampled to month-end
      4) SHY returns (if available in your 'rets')
      5) Constant 4%/yr as last resort
    """
    import pandas as pd, numpy as np
    from datetime import datetime

    def to_monthly_simple_from_annual_rate(s):  # s: monthly annualized rate in decimal
        # convert annualized % to monthly simple return
        return (1 + s/12) - 1

    # 1) Try FRED TB3MS
    try:
        from fredapi import Fred
        if fred_api_key in (None, "", "YOUR_FRED_API_KEY_HERE"):
            raise RuntimeError("No FRED API key provided")
        fred = Fred(api_key=fred_api_key)

        tb3 = fred.get_series("TB3MS")  # monthly, in percent (annualized)
        tb3 = tb3.to_frame("TB3MS").dropna()
        tb3 = tb3.resample("ME").last() / 100.0
        rf = to_monthly_simple_from_annual_rate(tb3["TB3MS"]).rename("RF")
        if rets_index is not None:
            rf = rf.reindex(rets_index).ffill().bfill()
        return rf
    except Exception as e:
        print(f"[warn] TB3MS from FRED failed ({e}). Trying DTB3...")

    # 2) Try FRED DTB3 (daily, 13-week bill)
    try:
        from fredapi import Fred
        if fred_api_key in (None, "", "YOUR_FRED_API_KEY_HERE"):
            raise RuntimeError("No FRED API key provided")
        fred = Fred(api_key=fred_api_key)

        dtb3 = fred.get_series("DTB3")  # daily % annualized
        dtb3 = dtb3.to_frame("DTB3").dropna()
        dtb3 = dtb3.resample("ME").last() / 100.0
        rf = to_monthly_simple_from_annual_rate(dtb3["DTB3"]).rename("RF")
        if rets_index is not None:
            rf = rf.reindex(rets_index).ffill().bfill()
        return rf
    except Exception as e:
        print(f"[warn] DTB3 from FRED failed ({e}). Trying Yahoo ^IRX...")

    # 3) Yahoo ^IRX (13-week T-bill index)
    try:
        import yfinance as yf
        irx = yf.download("^IRX", interval="1mo", auto_adjust=False, progress=False)["Adj Close"]
        # ^IRX is quoted in yield *percent* annualized
        irx = irx.to_frame("IRX").dropna()
        irx = irx.resample("ME").last() / 100.0
        rf = to_monthly_simple_from_annual_rate(irx["IRX"]).rename("RF")
        if rets_index is not None:
            rf = rf.reindex(rets_index).ffill().bfill()
        return rf
    except Exception as e:
        print(f"[warn] Yahoo ^IRX failed ({e}). Trying SHY proxy...")

    # 4) SHY proxy (if you already downloaded SHY monthly returns into `rets`)
    try:
        # expects a global `rets` DataFrame with a "SHY" column of monthly returns
        global rets
        if "SHY" in rets.columns:
            rf = rets["SHY"].copy()
            if rets_index is not None:
                rf = rf.reindex(rets_index).ffill().bfill()
            return rf.rename("RF")
    except Exception:
        pass

    # 5) Last resort: constant 4% annual -> monthly
    print("[warn] Falling back to constant 4%/yr RF.")
    rf = pd.Series(0.04/12, index=rets_index, name="RF")
    return rf

def get_gsci_tr_monthly():
    # S&P GSCI Total Return on FRED (SPGSCITR) — index level; convert to returns
    fred = Fred(api_key=FRED_API_KEY)
    gsci = fred.get_series("SPGSCITR").rename("SPGSCITR").dropna()
    gsci = gsci.resample("M").last()
    gsci_ret = gsci.pct_change().dropna().rename("GSCI_TR")
    return gsci_ret

def get_gold_monthly():
    # LBMA Gold Price (USD) daily -> monthly; FRED series GOLDAMGBD228NLBM (AM fix)
    fred = Fred(api_key=FRED_API_KEY)
    gold = fred.get_series("GOLDAMGBD228NLBM").rename("GOLD").dropna()
    gold = gold.replace(0, np.nan).dropna()
    gold = gold.resample("M").last()
    gold_ret = gold.pct_change().dropna().rename("GOLD")
    return gold_ret

def make_total_return(series_dict):
    """Align and build monthly return frame from provided return series."""
    df = pd.concat(series_dict.values(), axis=1).dropna(how="all")
    return df

# --- Download building blocks ---
rf = get_rf_monthly()

# Equities:
spx_tr = dl_yf("^SP500TR")                           # US stocks total return
vt_px  = dl_yf("VT")                                 # Global stocks (2008+); we will splice
# Bonds:
vbmfx_px = dl_yf("VBMFX")                            # US Aggregate (mutual fund) - long history
# REITs:
iyr_px = dl_yf("IYR")                                # US REITs (2000+)
# Commodities:
gsci_ret = get_gsci_tr_monthly()                     # Broad commodities TR
gold_ret = get_gold_monthly()                        # Gold
# Crypto:
btc_px = dl_yf("BTC-USD", interval="1mo")            # BTC (2014+ from Yahoo; treated 0% return before)

# Convert to returns
spx_ret = monthly_returns_from_prices(spx_tr.to_frame())["Adj Close"].rename("SPX_TR") if not spx_tr.empty else pd.Series(dtype=float, name="SPX_TR")
vt_ret  = monthly_returns_from_prices(vt_px.to_frame())["Adj Close"].rename("VT")      if not vt_px.empty else pd.Series(dtype=float, name="VT")
vbmfx_ret = monthly_returns_from_prices(vbmfx_px.to_frame())["Adj Close"].rename("AGG") if not vbmfx_px.empty else pd.Series(dtype=float, name="AGG")
iyr_ret = monthly_returns_from_prices(iyr_px.to_frame())["Adj Close"].rename("REIT")    if not iyr_px.empty else pd.Series(dtype=float, name="REIT")
btc_ret = monthly_returns_from_prices(btc_px.to_frame())["Adj Close"].rename("BTC")     if not btc_px.empty else pd.Series(dtype=float, name="BTC")

# Splice Global Equities: use VT where available; otherwise fall back to SPX_TR (note: not perfect)
global_equity = vt_ret.copy()
if global_equity.empty:
    global_equity = spx_ret.rename("GLOBAL")
else:
    # Pre-inception backfill with SPX_TR
    global_equity = pd.concat([spx_ret.rename("GLOBAL"), global_equity.rename("GLOBAL")], axis=1).ffill(axis=1).iloc[:, -1]
    global_equity.name = "GLOBAL"

# MBS proxy: if VMBS is unavailable historically, just use AGG (vbmfx_ret)
mbs_ret = vbmfx_ret.rename("MBS")

# Commodities blend (DBC + GLD in your weights): we’ll keep them separate so we can weight per your CAPM split
commods_ret = gsci_ret.rename("GSCI_TR")
gold_ret = gold_ret

# Cash monthly ret = rf
cash_ret = rf.rename("CASH")

# Align everything
rets = make_total_return({
    "SPX_TR": spx_ret,
    "GLOBAL": global_equity,
    "AGG": vbmfx_ret,
    "MBS": mbs_ret,
    "REIT": iyr_ret,
    "GSCI_TR": commods_ret,
    "GOLD": gold_ret,
    "BTC": btc_ret,
    "CASH": cash_ret
}).loc[START:]

# Fill BTC before it existed with 0% (i.e., no exposure growth pre-availability)
if "BTC" in rets.columns:
    first_btc = rets["BTC"].first_valid_index()
    if first_btc is not None:
        pre = rets.index < first_btc
        rets.loc[pre, "BTC"] = 0.0

# Helper: portfolio equity curve from weights
def portfolio_returns(weights):
    # weights is dict mapping our columns to weights (sum to 1)
    cols = [c for c in weights.keys() if c in rets.columns]
    w = pd.Series({c: weights[c] for c in cols})
    # forward-fill missing component returns with 0 (asset absent = no move)
    df = rets[cols].copy().fillna(0.0)
    port = (df * w.reindex(df.columns)).sum(axis=1)
    return port.rename("PORT")

def lever(ret, rf_series, L):
    # Futures-style leverage: Levered = L*ret + (1-L)*rf  (borrowing at rf for L>1)
    rf_al = rf_series.reindex(ret.index).fillna(method="ffill").fillna(0.0)
    return (L*ret + (1-L)*rf_al).rename(f"{ret.name}_x{L:.0f}")

def stats(ret, rf_series):
    m = ret.dropna()
    if len(m) < 12:
        return {"CAGR": np.nan, "Vol": np.nan, "Sharpe": np.nan, "MaxDD": np.nan, "FinalMultiple": np.nan}
    years = len(m)/12
    curve = (1+m).cumprod()
    cagr = curve.iloc[-1]**(1/years) - 1
    vol = m.std(ddof=1)*np.sqrt(12)
    rf_m = rf_series.reindex(m.index).fillna(method="ffill").fillna(0.0)
    ex = m - rf_m
    sharpe = (ex.mean()*12)/(ex.std(ddof=1)*np.sqrt(12)) if ex.std(ddof=1)>0 else np.nan
    dd = (curve/curve.cummax() - 1).min()
    return {"CAGR": cagr, "Vol": vol, "Sharpe": sharpe, "MaxDD": dd, "FinalMultiple": curve.iloc[-1]}

def make_rows(name, base_ret, rf_series):
    rows = []
    for L in [1,2,3]:
        lr = lever(base_ret, rf_series, L)
        st = stats(lr, rf_series)
        st["Portfolio"] = name
        st["Leverage"] = f"{L}x"
        rows.append(st)
    return rows

# ---------- Define portfolios ----------
# 1) CAPM-style from your weights (tweakable mapping)
# Global equities VT 44%, Global bonds BND,VMBS 34%, Real estate VNQ,VNQI 7%,
# Commodities DBC,GLD 5%, Crypto 2%, Cash 8%.
capm_weights = {
    "GLOBAL": 0.44,                   # VT proxy
    "AGG": 0.27, "MBS": 0.07,         # split 34% into core bond + MBS proxy
    "REIT": 0.07,                     # REITs
    "GSCI_TR": 0.025, "GOLD": 0.025,  # 5% split half/half
    "BTC": 0.02,                      # crypto
    "CASH": 0.08                      # cash buffer
}

# 2) 60/40 (stocks/bonds) — use US stocks and US Agg for simplicity
sixty_forty = {
    "SPX_TR": 0.60,
    "AGG": 0.40
}

# 3) ALLW (All Weather–ish): approximate (can tweak)
# Example: 30% stocks, 55% bonds (mix of Agg & long), 7.5% gold, 7.5% commodities
# We don't have long bonds series here, so keep bonds in AGG for robustness; add REIT small slice.
allw = {
    "SPX_TR": 0.30,
    "AGG": 0.55,
    "GOLD": 0.075,
    "GSCI_TR": 0.075
}

# 4) VT (global stocks only)
vt_only = {"GLOBAL": 1.0}

# 5) VOO (US large cap)
voo_only = {"SPX_TR": 1.0}

port_defs = [
    ("CAPM", capm_weights),
    ("60/40", sixty_forty),
    ("ALLW", allw),
    ("VT", vt_only),
    ("VOO", voo_only),
]

# ---------- Compute ----------
rf_aligned = rf.reindex(rets.index).fillna(method="ffill").fillna(0.0)
all_rows = []
curves = []

for name, w in port_defs:
    base = portfolio_returns(w)
    base.name = name
    curves.append((1+base).cumprod().rename(f"{name}_1x"))
    for L in [1,2,3]:
        lr = lever(base, rf_aligned, L)
        curves.append((1+lr).cumprod().rename(f"{name}_{L}x"))
        all_rows.extend(make_rows(name, base, rf_aligned))

results = pd.DataFrame(all_rows)[["Portfolio","Leverage","CAGR","Vol","Sharpe","MaxDD","FinalMultiple"]]
results = results.sort_values(["Portfolio","Leverage"]).reset_index(drop=True)

# ---------- Save & Plot ----------
def fmt_pct(x): 
    return f"{x:.2%}" if isinstance(x, (float,np.floating)) else x

out_curves = pd.concat(curves, axis=1).dropna()
results.to_csv("portfolio_results.csv", index=False)
out_curves.to_csv("equity_curves.csv")

print("\n=== Backtest window:", out_curves.index[0].date(), "→", out_curves.index[-1].date(), "===")
print(results.assign(
    CAGR=lambda d: d["CAGR"].map(fmt_pct),
    Vol=lambda d: d["Vol"].map(fmt_pct),
    Sharpe=lambda d: d["Sharpe"].map(lambda v: f"{v:.2f}" if pd.notna(v) else "NA"),
    MaxDD=lambda d: d["MaxDD"].map(fmt_pct),
    FinalMultiple=lambda d: d["FinalMultiple"].map(lambda v: f"{v:.2f}x")
).to_string(index=False))

plt.figure(figsize=(11,6))
for col in out_curves.columns:
    plt.plot(out_curves.index, out_curves[col], label=col)
plt.yscale("log")
plt.title("Equity Curves (log scale)")
plt.legend(ncol=3, fontsize=8)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("\nSaved:")
print(" - portfolio_results.csv")
print(" - equity_curves.csv")


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject